In [1]:
!pip install langchain_openai
!pip install langchain_chroma
!pip install langchain_groq
!pip install langchain_community
!pip install langchain_core
!pip install langchain_ray
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from langchain_community.vectorstores import chroma
from langchain_community.llms import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
import ray
import os
import dask.dataframe as dd

In [3]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("CUDA device name:", torch.cuda.get_device_name(0))
    print("Current device index:", torch.cuda.current_device())

CUDA available: True
CUDA device count: 1
CUDA device name: NVIDIA A100-SXM4-40GB
Current device index: 0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
dataframe = pd.read_csv("filtered_data.csv")

In [9]:
len(dataframe)

53872

In [10]:
catgories = list(dataframe["Crop"].unique())

In [11]:
catgories

['Apple', 'Coconut', 'Paddy Dhan', 'Potato', 'Tomato']

In [12]:
!pip install langchain_huggingface

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1",model_kwargs={"device": "cuda"})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
import numpy as np

## Mapping the season names to its corresponding English names

In [16]:
season_map = {
    'JAYAD': 'Residual Crop',
    'RABI': 'Winter Crop',
    'KHARIF': 'Monsoon Crop',
}

In [17]:
dataframe["Season"].unique()

array(['JAYAD', nan, 'RABI', 'KHARIF', '0'], dtype=object)

In [18]:
dataframe['season_clean'] = dataframe['Season'].replace({np.nan: "", '0': ""})

In [19]:
dataframe['Season_English'] = dataframe['season_clean'].map(season_map)

In [20]:
from tqdm import tqdm

In [21]:
dataframe['Season_English'].unique()

array(['Residual Crop', nan, 'Winter Crop', 'Monsoon Crop'], dtype=object)

## Renaming Paddy Dhan to Paddy_Dhan

In [29]:
dataframe['Crop'] = dataframe['Crop'].replace('Paddy Dhan', 'Paddy_Dhan')

In [30]:
dataframe['Crop'].unique()

array(['Apple', 'Coconut', 'Paddy_Dhan', 'Potato', 'Tomato'], dtype=object)

## Getting the Unique categories

In [33]:
catgories = list(dataframe["Crop"].unique())

In [34]:
category_groups = {cat: dataframe[dataframe['Crop'] == cat] for cat in catgories}

In [35]:
category_groups.keys()

dict_keys(['Apple', 'Coconut', 'Paddy_Dhan', 'Potato', 'Tomato'])

## Creating the Vector Database

In [36]:
for cat, df_cat in tqdm(category_groups.items(),desc="Processing categories"):

    print(len(df_cat))
    docs = [
        Document(page_content=f"DistrictName:{row['DistrictName']}\nStateName:{row['StateName']}\nSeason_English:{row['Season_English']}\nMonth:{row['Month']}\nDisease:{row['disease_name']}\nQueryText:{row['QueryText']}\nKccAns:{row['KccAns']}",
                 metadata={"disease_name": row['disease_name'],"StateName":row["StateName"]})
        for _, row in df_cat.iterrows()
    ]

    chroma_collection = Chroma.from_documents(
        docs,
        embedding=embedding,
        persist_directory="./chroma_capstone_db_new",
        collection_name=cat
    )

Processing categories:   0%|          | 0/5 [00:00<?, ?it/s]

2390


Processing categories:  20%|██        | 1/5 [00:02<00:10,  2.57s/it]

3672


Processing categories:  40%|████      | 2/5 [00:06<00:09,  3.29s/it]

...

Processing categories:  60%|██████    | 3/5 [00:45<00:39, 19.57s/it]

5937


Processing categories:  80%|████████  | 4/5 [00:51<00:14, 14.42s/it]

6577


Processing categories: 100%|██████████| 5/5 [00:58<00:00, 11.76s/it]


In [38]:
chroma_db = Chroma(
    persist_directory="./chroma_capstone_db_new",
    embedding_function=embedding,
    collection_name="Tomato"  # Specify which collection to load
)

In [39]:
question = "give me the cure for tomato plant"
docs = chroma_db.max_marginal_relevance_search(question, k=3, fetch_k=6)